# Data preprocessing

In [1]:
import pandas
import numpy as np
import sklearn.linear_model as lm
from sklearn.model_selection import KFold
from sklearn import preprocessing as pre
import random
import matplotlib.pyplot as plt



## Reading data

In [2]:
housing = pandas.read_csv('./housing.csv')

## Transform ocean_proximity column to 5 binary-valued columns

In [3]:
housing['1h_ocean'] = [1 if i=='<1H OCEAN' else 0 for i in housing.ocean_proximity.values]
housing['island'] = [1 if i=='ISLAND' else 0 for i in housing.ocean_proximity.values]
housing['inland'] = [1 if i=='INLAND' else 0 for i in housing.ocean_proximity.values]
housing['near_ocean'] = [1 if i=='NEAR OCEAN' else 0 for i in housing.ocean_proximity.values]
housing['near_bay'] = [1 if i=='NEAR BAY' else 0 for i in housing.ocean_proximity.values]
housing.drop(columns=['ocean_proximity'], inplace=True)

## Filling missing data

Missing total_bedroom values are predicted through linear regression with total_rooms values

In [4]:
notna = housing.total_bedrooms.notna()
model = lm.LinearRegression()
model.fit(housing.total_rooms.values[notna].reshape(-1,1), housing.total_bedrooms.values[notna].reshape(-1,1))
isna = housing.total_bedrooms.isna()
missing_bedrooms = model.predict(housing.total_rooms.values[isna].reshape(-1,1))
housing.total_bedrooms.loc[isna] = np.squeeze(missing_bedrooms)

c:\users\alun\appdata\local\programs\python\python37\lib\site-packages\pandas\core\indexing.py:189: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/indexing.html#indexing-view-versus-copy
  self._setitem_with_indexer(indexer, value)


## The code below is for feature scaling (normalisation):

In [ ]:
import pandas
import numpy as np
from sklearn.model_selection import KFold
import random
import sklearn.linear_model as lm
from sklearn import preprocessing
import DataPrepUtil
import Impute


housing = pandas.read_csv('housing.csv')

DataPrepUtil.transform_ocean_proximity(housing)
Impute.fill_lr_prediction_from_other_column(housing, 'total_rooms')


scaled_1 = pandas.DataFrame(preprocessing.StandardScaler().fit_transform(housing))
scaled_2 = pandas.DataFrame(preprocessing.MaxAbsScaler().fit_transform(housing))
scaled_3 = pandas.DataFrame(preprocessing.MinMaxScaler().fit_transform(housing))
scaled_4 = pandas.DataFrame(preprocessing.RobustScaler().fit_transform(housing))

scaled_1.columns = housing.columns
scaled_2.columns = housing.columns
scaled_3.columns = housing.columns
scaled_4.columns = housing.columns

scaler_names = ['StandardScaler', 'MaxAbsScaler', 'MinMaxScaler', 'RobustScaler'] 

scaler_count = 0

for i in (scaled_1, scaled_2, scaled_3, scaled_4): 
    
 
    housing = i
    
    y = housing.median_house_value.values.reshape(-1,1)
    X = housing.drop(columns=['median_house_value'], inplace=False).values
    
    holdout = random.sample(range(0,10640),1000)
    X_holdout = X[holdout]
    y_holdout = y[holdout]
    Xt = np.delete(X, holdout, 0)
    yt = np.delete(y, holdout, 0)
    
    fold_number = 10
    
    kf = KFold(n_splits=fold_number, shuffle=True)
    
    Model = lm.LinearRegression()

    train_R2_average = 0
    test_R2_average = 0
    mae_average = 0
    accuracy_average = 0
        
    for train_index, test_index in kf.split(Xt): 
        X_train, X_test = Xt[train_index], Xt[test_index]
        y_train, y_test = yt[train_index], yt[test_index]
        
        Model.fit(X_train, y_train.ravel()) 
        
        
        train_R2_average += Model.score(X_train, y_train)/fold_number
        test_R2_average += Model.score(X_test, y_test)/fold_number

        
    print('Average training R^2 score for ', scaler_names[scaler_count], ': ', train_R2_average)
    
    print('Average test R^2 scorefor ', scaler_names[scaler_count], ': ', test_R2_average, '\n')
    scaler_count += 1
    


## Data Transform using Box Cox Transform

In [5]:
from scipy import stats

median_house_value_bc, maxlog, interval = stats.boxcox(housing.median_house_value, alpha=0.05)
population_bc, maxlog, interval = stats.boxcox(housing.population, alpha=0.05)
housing_median_age_bc, maxlog, interval = stats.boxcox(housing.housing_median_age, alpha=0.05)
total_rooms_bc, maxlog, interval = stats.boxcox(housing.total_rooms, alpha=0.05)
total_bedrooms_bc, maxlog, interval = stats.boxcox(housing.total_bedrooms, alpha=0.05)
households_bc, maxlog, interval = stats.boxcox(housing.households, alpha=0.05)
median_income_bc, maxlog, interval = stats.boxcox(housing.median_income, alpha=0.05)



In [6]:
housing_boxcox = housing.copy()

housing_boxcox.drop(columns=['housing_median_age'], inplace=True)
housing_boxcox.drop(columns=['total_rooms'], inplace=True)
housing_boxcox.drop(columns=['total_bedrooms'], inplace=True)
housing_boxcox.drop(columns=['population'], inplace=True)
housing_boxcox.drop(columns=['households'], inplace=True)
housing_boxcox.drop(columns=['median_income'], inplace=True)
housing_boxcox.drop(columns=['median_house_value'], inplace=True)

housing_boxcox['housing_median_age'] = housing_median_age_bc
housing_boxcox['total_rooms'] = total_rooms_bc
housing_boxcox['total_bedrooms'] = total_bedrooms_bc
housing_boxcox['population'] = population_bc
housing_boxcox['households'] = households_bc
housing_boxcox['median_income'] = median_income_bc
housing_boxcox['median_house_value'] = median_house_value_bc


print(housing_boxcox)

       longitude  latitude  1h_ocean  island  inland  near_ocean  near_bay  \
0        -122.23     37.88         0       0       0           0         1   
1        -122.22     37.86         0       0       0           0         1   
2        -122.24     37.85         0       0       0           0         1   
3        -122.25     37.85         0       0       0           0         1   
4        -122.25     37.85         0       0       0           0         1   
5        -122.25     37.85         0       0       0           0         1   
6        -122.25     37.84         0       0       0           0         1   
7        -122.25     37.84         0       0       0           0         1   
8        -122.26     37.84         0       0       0           0         1   
9        -122.25     37.84         0       0       0           0         1   
10       -122.26     37.85         0       0       0           0         1   
11       -122.26     37.85         0       0       0           0

 ## Support Vector Machines
 


The code below simply runs testsing code found in support_vector_machines.py

In [ ]:
import support_vector_machines

cross_set, eval_set = support_vector_machines.create_kfold_sets(housing_boxcox)
support_vector_machines.run_svr(cross_set, eval_set,c=10**6)

## After data transform, please use housing_boxcox as input

In [ ]:
# First, extract the data into arrays
y = housing_boxcox.median_house_value.values.reshape(-1,1)
X = housing_boxcox.drop(columns=['median_house_value'], inplace=False).values

print(X.shape)
print(y.shape)
# Pull out 1000 values into a holdout set
holdout = random.sample(range(0,10640),1000)
X_holdout = X[holdout]
y_holdout = y[holdout]
Xt = np.delete(X, holdout, 0)
yt = np.delete(y, holdout, 0)
print(Xt.shape)
print(yt.shape)

train_R2_average = 0
test_R2_average = 0
mae_average = 0
accuracy_average = 0

fold_number = 10

# Have to shuffle the data because it is grouped.
kf = KFold(n_splits=fold_number, shuffle=True)
for train_index, test_index in kf.split(Xt):
  X_train, X_test = Xt[train_index], Xt[test_index]
  y_train, y_test = yt[train_index], yt[test_index]
  Model.fit(X_train, y_train)
  predictions = Model.predict(X_test)
  errors = abs(predictions - y_test)
  mae = round(numpy.mean(errors), 2)
  mae_average += mae/fold_number
  print('Mean Absolute Error:', mae)
  mape = 100 * (errors / y_test)
  accuracy = 100 - numpy.mean(mape)
  accuracy_average += accuracy/fold_number
  print('Accuracy:', round(accuracy, 2), '%.')
  print('Training R^2: ' , Model.score(X_train, y_train))
  print('Testing R^2: ' , Model.score(X_test, y_test))
  print()
  train_R2_average += Model.score(X_train, y_train)/fold_number
  test_R2_average += Model.score(X_test, y_test)/fold_number
print('Average train accuracy: ', accuracy_average)
print('Average train mae: ', mae_average)
print('Average train R^2: ', train_R2_average)
print('Average test R^2: ', test_R2_average)